In [39]:
import random
import simpy

class VM:
    def __init__(self, id, cpu, ram):
        self.id = id
        self.cpu = cpu
        self.ram = ram
        self.cost = cpu / 10
        self.available_cpu = cpu
        self.available_ram = ram

class Task:
    def __init__(self, id, instructions, memory, deadline):
        self.id = id
        self.instructions = instructions
        self.memory = memory
        self.deadline = deadline

In [2]:
def round_robin_allocate(vms, tasks):
    allocations = []
    vm_index = 0
    num_vms = len(vms)
    
    for task in tasks:
        vm_id = vm_index % num_vms
        allocations.append(vm_id)
        vm_index += 1
    
    return allocations

def rr_optimize(vms, tasks):
    allocations = round_robin_allocate(vms, tasks)
    return run_simulation(allocations, vms, tasks)

In [3]:
def sjf_allocate(vms, tasks):
    sorted_tasks = sorted(enumerate(tasks), key=lambda x: x[1].instructions)
    allocations = [0] * len(tasks)
    vm_loads = [0] * len(vms)
    
    for task_index, task in sorted_tasks:
        vm_id = min(range(len(vms)), key=lambda i: vm_loads[i])
        allocations[task_index] = vm_id
        vm_loads[vm_id] += task.instructions
    
    return allocations

def sjf_optimize(vms, tasks):
    allocations = sjf_allocate(vms, tasks)
    return run_simulation(allocations, vms, tasks)

In [4]:
def calculate_overutilization_penalty(vm_utilization, vms):
    penalty = 0
    for vm in vms:
        cpu_overuse = max(0, vm_utilization[vm.id]['cpu'] - vm.cpu)
        ram_overuse = max(0, vm_utilization[vm.id]['ram'] - vm.ram)
        penalty += (cpu_overuse / vm.cpu + ram_overuse / vm.ram) * 1000  # Adjust multiplier as needed
    return penalty

In [5]:
def flexible_allocate_tasks(vm_preferences, vms, tasks):
    allocations = []
    vm_utilization = {vm.id: {'cpu': 0, 'ram': 0} for vm in vms}
    
    for task_id, preferred_vm_id in enumerate(vm_preferences):
        task = tasks[task_id]
        allocated_cpu = 0
        allocated_ram = 0
        task_allocations = []

        # Try to allocate to the preferred VM first
        preferred_vm = vms[preferred_vm_id]
        alloc_cpu = min(task.instructions - allocated_cpu, preferred_vm.cpu - vm_utilization[preferred_vm.id]['cpu'])
        alloc_ram = min(task.memory - allocated_ram, preferred_vm.ram - vm_utilization[preferred_vm.id]['ram'])
        
        if alloc_cpu > 0 or alloc_ram > 0:
            task_allocations.append((preferred_vm.id, alloc_cpu, alloc_ram))
            vm_utilization[preferred_vm.id]['cpu'] += alloc_cpu
            vm_utilization[preferred_vm.id]['ram'] += alloc_ram
            allocated_cpu += alloc_cpu
            allocated_ram += alloc_ram

        # If task is not fully allocated, try other VMs
        for vm in vms:
            if allocated_cpu >= task.instructions and allocated_ram >= task.memory:
                break
            if vm.id == preferred_vm.id:
                continue
            
            alloc_cpu = min(task.instructions - allocated_cpu, vm.cpu - vm_utilization[vm.id]['cpu'])
            alloc_ram = min(task.memory - allocated_ram, vm.ram - vm_utilization[vm.id]['ram'])
            
            if alloc_cpu > 0 or alloc_ram > 0:
                task_allocations.append((vm.id, alloc_cpu, alloc_ram))
                vm_utilization[vm.id]['cpu'] += alloc_cpu
                vm_utilization[vm.id]['ram'] += alloc_ram
                allocated_cpu += alloc_cpu
                allocated_ram += alloc_ram

        allocations.append(task_allocations)

    return allocations, vm_utilization

In [6]:
def run_simulation(vm_preferences, vms, tasks):
    allocations, vm_utilization = flexible_allocate_tasks(vm_preferences, vms, tasks)
    
    total_cost = 0
    total_energy = 0
    max_execution_time = 0
    
    for task_id, task_allocations in enumerate(allocations):
        task = tasks[task_id]
        task_execution_time = 0
        for vm_id, alloc_cpu, alloc_ram in task_allocations:
            vm = next(vm for vm in vms if vm.id == vm_id)
            execution_time = alloc_cpu / vm.cpu * 1000  # ms
            task_execution_time += execution_time
            
            # Calculate cost based on resource usage and time
            cost = vm.cost * (execution_time / 1000) * (alloc_cpu / vm.cpu)
            total_cost += cost
            
            # Calculate energy based on CPU utilization and time
            energy = vm.cpu * 0.001 * (alloc_cpu / vm.cpu) * (execution_time / 1000)
            total_energy += energy
        
        max_execution_time = max(max_execution_time, task_execution_time)
    
    overutilization_penalty = calculate_overutilization_penalty(vm_utilization, vms)
    
    return max_execution_time, total_cost, total_energy, overutilization_penalty

def evaluate(individual, vms, tasks):
    return run_simulation(individual, vms, tasks)

In [7]:
def check_fitness(individual):
    makespan, cost, energy, overutilization_penalty = run_simulation(individual, vms, tasks)
    
    # Normalize and combine objectives
    normalized_makespan = makespan / 1000
    normalized_cost = cost / 100
    normalized_energy = energy / 100

    weighted_sum = (normalized_makespan * 0.4 + 
                    normalized_cost * 0.3 + 
                    normalized_energy * 0.2 + 
                    overutilization_penalty * 0.1)
    
    return weighted_sum, makespan, cost, energy, overutilization_penalty

In [8]:
#FWA
def fwa_optimize(vms,tasks):
    def generate_sparks(firework, n, amplitude, num_vms):
        return [
            [max(0, min(num_vms - 1, x + int(random.uniform(-amplitude, amplitude))))
             for x in firework]
            for _ in range(n)
        ]
    def fireworks_algorithm(vms, tasks, n_fireworks=5, n_sparks=30, n_iterations=40):
        num_vms = len(vms)
        num_tasks = len(tasks)

        # Initialize fireworks
        fireworks = [random.choices(range(num_vms), k=num_tasks) for _ in range(n_fireworks)]
        best_solution = None
        best_fitness = float('inf')

        for _ in range(n_iterations):
            all_solutions = fireworks.copy()

            # Generate sparks
            for firework in fireworks:
                fitness = check_fitness(firework)[0]
                if fitness < best_fitness:
                    best_fitness = fitness
                    best_solution = firework

                # Calculate amplitude, ensuring it's within a reasonable range
                amplitude = max(1, int(50 * best_fitness / fitness))

                all_solutions.extend(generate_sparks(firework, n_sparks, amplitude, num_vms))

            # Evaluate and select best solutions
            all_solutions.sort(key=lambda x: check_fitness(x)[0])
            fireworks = all_solutions[:n_fireworks]

        return best_solution, best_fitness

    best_solution, best_fitness = fireworks_algorithm(vms, tasks)
    return evaluate(best_solution, vms, tasks)

In [9]:
#SQSA
def sqsa_optimize(vms, tasks, n_squirrels=30, n_iterations=40):
    squirrels = [random.choices(range(len(vms)), k=len(tasks)) for _ in range(n_squirrels)]
    fitness = [check_fitness(s)[0] for s in squirrels]

    best_idx = fitness.index(min(fitness))
    
    for _ in range(n_iterations):
        for i in range(n_squirrels):
            if i != best_idx:
                for j in range(len(tasks)):
                    if random.random() < 0.5:
                        squirrels[i][j] = squirrels[best_idx][j]
                    else:
                        squirrels[i][j] = random.randint(0, len(vms)-1)
        
        fitness = [check_fitness(s)[0] for s in squirrels]
        new_best_idx = fitness.index(min(fitness))
        if fitness[new_best_idx] < fitness[best_idx]:
            best_idx = new_best_idx

    return evaluate(squirrels[best_idx], vms, tasks)

In [10]:
# BAT
def bat_optimize(vms, tasks, n_bats=30, n_iterations=40):
    bats = [random.choices(range(len(vms)), k=len(tasks)) for _ in range(n_bats)]
    velocities = [[0] * len(tasks) for _ in range(n_bats)]
    fitness = [check_fitness(b)[0] for b in bats]
    best_bat = bats[fitness.index(min(fitness))]
    
    for _ in range(n_iterations):
        for i in range(n_bats):
            freq = random.random()
            for j in range(len(tasks)):
                velocities[i][j] += (bats[i][j] - best_bat[j]) * freq
                bats[i][j] = max(0, min(len(vms)-1, int(bats[i][j] + velocities[i][j])))
            
            if random.random() > 0.5:
                bats[i] = [max(0, min(len(vms)-1, best_bat[j] + int(random.gauss(0, 1))))
                           for j in range(len(tasks))]
        
        fitness = [check_fitness(b)[0] for b in bats]
        best_idx = fitness.index(min(fitness))
        if fitness[best_idx] < check_fitness(best_bat)[0]:
            best_bat = bats[best_idx]
    return evaluate(best_bat, vms, tasks)

In [11]:
# PSO
def pso_optimize(vms, tasks, n_particles=30, n_iterations=40):
    particles = [random.choices(range(len(vms)), k=len(tasks)) for _ in range(n_particles)]
    velocities = [[0] * len(tasks) for _ in range(n_particles)]
    personal_best = particles.copy()
    fitness = [check_fitness(p)[0] for p in particles]
    global_best = particles[fitness.index(min(fitness))]
    
    for _ in range(n_iterations):
        for i in range(n_particles):
            for j in range(len(tasks)):
                r1, r2 = random.random(), random.random()
                velocities[i][j] = (0.5 * velocities[i][j] + 
                                    1 * r1 * (personal_best[i][j] - particles[i][j]) +
                                    2 * r2 * (global_best[j] - particles[i][j]))
                particles[i][j] = max(0, min(len(vms)-1, int(particles[i][j] + velocities[i][j])))
            
            current_fitness = check_fitness(particles[i])[0]

            if current_fitness < check_fitness(personal_best[i])[0]:
                personal_best[i] = particles[i]
                if current_fitness < check_fitness(global_best)[0]:
                    global_best = particles[i]

    return evaluate(global_best, vms, tasks)

In [12]:
# BMO
def bmo_optimize(vms, tasks, n_barnacles=30, n_iterations=40):
    barnacles = [random.choices(range(len(vms)), k=len(tasks)) for _ in range(n_barnacles)]
    fitness = [check_fitness(b)[0] for b in barnacles]
    best_barnacle = barnacles[fitness.index(min(fitness))]
    
    for _ in range(n_iterations):
        for i in range(n_barnacles):
            if random.random() < 0.5:  # Pseudo-fertilization
                partner = random.choice(barnacles)
                barnacles[i] = [b if random.random() < 0.5 else p 
                                for b, p in zip(barnacles[i], partner)]
            else:  # Larval development
                barnacles[i] = [max(0, min(len(vms)-1, x + int(random.gauss(0, 1))))
                                for x in barnacles[i]]
        
        fitness = [check_fitness(b)[0] for b in barnacles]
        new_best = barnacles[fitness.index(min(fitness))]
        if check_fitness(new_best)[0] < check_fitness(best_barnacle)[0]:
            best_barnacle = new_best

    return evaluate(best_barnacle, vms, tasks)

In [13]:
#SSA
def ssa_optimize(vms, tasks, n_sparrows=30, n_iterations=40):
    sparrows = [random.choices(range(len(vms)), k=len(tasks)) for _ in range(n_sparrows)]
    fitness = [check_fitness(s)[0] for s in sparrows]
    best_sparrow = sparrows[fitness.index(min(fitness))]
    
    for _ in range(n_iterations):
        for i in range(n_sparrows):
            if i < n_sparrows // 5:  # Discoverer
                sparrows[i] = [max(0, min(len(vms)-1, x + int(random.gauss(0, 1))))
                               for x in sparrows[i]]
            elif i < n_sparrows // 2:  # Joiner
                sparrows[i] = [best_sparrow[j] if random.random() < 0.5 else sparrows[i][j]
                               for j in range(len(tasks))]
            else:  # Scrounger
                sparrows[i] = [random.randint(0, len(vms)-1) if random.random() < 0.1 else sparrows[i][j]
                               for j in range(len(tasks))]
        
        fitness = [check_fitness(s)[0] for s in sparrows]
        new_best = sparrows[fitness.index(min(fitness))]
        if check_fitness(new_best)[0] < check_fitness(best_sparrow)[0]:
            best_sparrow = new_best

    return evaluate(best_sparrow, vms, tasks)

In [25]:
num_vms = 30
num_tasks = 100
vms = [VM(i, random.randint(1000, 3000), random.randint(1000, 2000)) for i in range(num_vms)]
tasks = [Task(i, random.randint(100, 500), random.randint(100, 500), random.randint(5, 50)) for i in range(num_tasks)]

In [41]:
rr_makespan, rr_cost, rr_energy, rr_penalty = rr_optimize(vms, tasks)
rr_makespan, rr_cost, rr_energy

(436.3001745200698, 584.7653642159868, 5.847653642159868)

In [47]:
sjf_makespan, sjf_cost, sjf_energy, sjf_penalty = sjf_optimize(vms, tasks)
sjf_makespan, sjf_cost, sjf_energy

(395.6521739130435, 571.2980459294112, 5.712980459294113)

In [54]:
initial_allocation = random.choices(range(len(vms)), k=len(tasks))
makespan, total_cost, total_energy, overutilization_penalty = run_simulation(initial_allocation, vms, tasks)
makespan,total_cost,total_energy

(426.25745950554136, 531.882934180545, 5.31882934180545)

In [29]:
fwa_makespan, fwa_cost, fwa_energy,fwa_allocation_status=fwa_optimize(vms,tasks)
fwa_makespan,fwa_cost,fwa_energy

(282.01970443349757, 446.8222903247335, 4.468222903247337)

In [30]:
sqsa_makespan, sqsa_cost, sqsa_energy, sqsa_allocation_status=sqsa_optimize(vms,tasks)
sqsa_makespan, sqsa_cost, sqsa_energy

(288.1773399014778, 481.31610674148465, 4.81316106741485)

In [31]:
bat_makespan, bat_cost, bat_energy, bat_allocation_status=bat_optimize(vms,tasks)
bat_makespan, bat_cost, bat_energy

(282.6354679802955, 442.2459059649198, 4.422459059649201)

In [36]:
pso_makespan, pso_cost, pso_energy, pso_allocation_status=pso_optimize(vms,tasks)
pso_makespan, pso_cost, pso_energy

(375.10656436487636, 536.5051627351316, 5.365051627351319)

In [37]:
bmo_makespan, bmo_cost, bmo_energy, bmo_allocation_status = bmo_optimize(vms, tasks)
bmo_makespan, bmo_cost, bmo_energy

(348.92638036809814, 493.63900439399976, 4.936390043939995)

In [38]:
ssa_makespan, ssa_cost, ssa_energy, ssa_allocation_status = ssa_optimize(vms, tasks)
ssa_makespan, ssa_cost, ssa_energy

(305.6080655324512, 468.9260644841854, 4.689260644841853)

In [35]:
def compare_results(results):
    for name, (makespan, cost, energy) in results.items():
        print(f"{name}:")
        print(f"  Makespan: {makespan}")
        print(f"  Cost: {cost}")
        print(f"  Energy: {energy}")
        print()

    best_makespan = min(results.items(), key=lambda x: x[1][0])
    best_cost = min(results.items(), key=lambda x: x[1][1])
    best_energy = min(results.items(), key=lambda x: x[1][2])

    print(f"Best Makespan: {best_makespan[0]}")
    print(f"Best Cost: {best_cost[0]}")
    print(f"Best Energy: {best_energy[0]}")
    
# Collect results
results = {
    "No Optimization": (makespan, total_cost, total_energy),
    "Round Robin": (rr_makespan,rr_cost,rr_energy),
    "Shortest Job First": (sjf_makespan, sjf_cost, sjf_energy),
    "Fireworks Algorithm": (fwa_makespan, fwa_cost, fwa_energy),
    "Squirrel Search Algorithm": (sqsa_makespan,sqsa_cost,sqsa_energy),
    "Bat Algorithm": (bat_makespan, bat_cost, bat_energy),
    "Particle Swarm Optimizer": (pso_makespan, pso_cost, pso_energy),
    "Barnacles Mating Optimizer": (bmo_makespan, bmo_cost, bmo_energy),
    "Sparrow Search Algorithm": (ssa_makespan, ssa_cost, ssa_energy)
}

compare_results(results)

No Optimization:
  Makespan: 400.5235602094241
  Cost: 560.1746091485845
  Energy: 5.601746091485844

Round Robin:
  Makespan: 436.3001745200698
  Cost: 584.7653642159868
  Energy: 5.847653642159868

Shortest Job First:
  Makespan: 395.6521739130435
  Cost: 571.2980459294112
  Energy: 5.712980459294113

Fireworks Algorithm:
  Makespan: 282.01970443349757
  Cost: 446.8222903247335
  Energy: 4.468222903247337

Squirrel Search Algorithm:
  Makespan: 288.1773399014778
  Cost: 481.31610674148465
  Energy: 4.81316106741485

Bat Algorithm:
  Makespan: 282.6354679802955
  Cost: 442.2459059649198
  Energy: 4.422459059649201

Particle Swarm Optimizer:
  Makespan: 399.650959860384
  Cost: 519.5113090128201
  Energy: 5.1951130901282045

Barnacles Mating Optimizer:
  Makespan: 333.0727130570759
  Cost: 501.7845754092202
  Energy: 5.0178457540922

Sparrow Search Algorithm:
  Makespan: 347.29493891797557
  Cost: 493.536500044236
  Energy: 4.935365000442361

Best Makespan: Fireworks Algorithm
Best Cos

# Results->

vms will range from 10 to 50
tasks will range from 40 to 120

In [76]:
num_vms = 10
num_tasks = 40

In [77]:
rr_mean_makespan,rr_mean_energy,rr_mean_cost=0,0,0
sjf_mean_makespan,sjf_mean_energy,sjf_mean_cost=0,0,0
fwa_mean_makespan,fwa_mean_energy,fwa_mean_cost=0,0,0
sqsa_mean_makespan,sqsa_mean_energy,sqsa_mean_cost=0,0,0
bat_mean_makespan,bat_mean_energy,bat_mean_cost=0,0,0
pso_mean_makespan,pso_mean_energy,pso_mean_cost=0,0,0
bmo_mean_makespan,bmo_mean_energy,bmo_mean_cost=0,0,0
ssa_mean_makespan,ssa_mean_energy,ssa_mean_cost=0,0,0

In [78]:
for _ in range(10):
    vms = [VM(i, random.randint(1000, 3000), random.randint(1000, 2000)) for i in range(num_vms)]
    tasks = [Task(i, random.randint(100, 500), random.randint(100, 500), random.randint(5, 50)) for i in range(num_tasks)]
    
    # RR
    rr_makespan, rr_cost, rr_energy= rr_optimize(vms, tasks)[:3]
    rr_mean_makespan+=rr_makespan
    rr_mean_cost+=rr_cost
    rr_mean_energy+=rr_energy
    
    # SJF
    sjf_makespan, sjf_cost, sjf_energy= sjf_optimize(vms, tasks)[:3]
    sjf_mean_makespan+=sjf_makespan
    sjf_mean_cost+=sjf_cost
    sjf_mean_energy+=sjf_energy
    
    #FWA
    fwa_makespan, fwa_cost, fwa_energy= fwa_optimize(vms, tasks)[:3]
    fwa_mean_makespan+=fwa_makespan
    fwa_mean_cost+=fwa_cost
    fwa_mean_energy+=fwa_energy
    
    #SQSA
    sqsa_makespan, sqsa_cost, sqsa_energy= sqsa_optimize(vms, tasks)[:3]
    sqsa_mean_makespan+=sqsa_makespan
    sqsa_mean_cost+=sqsa_cost
    sqsa_mean_energy+=sqsa_energy

    #BAT
    bat_makespan, bat_cost, bat_energy= bat_optimize(vms, tasks)[:3]
    bat_mean_makespan+=bat_makespan
    bat_mean_cost+=bat_cost
    bat_mean_energy+=bat_energy
    
    #PSO
    pso_makespan, pso_cost, pso_energy= pso_optimize(vms, tasks)[:3]
    pso_mean_makespan+=pso_makespan
    pso_mean_cost+=pso_cost
    pso_mean_energy+=pso_energy

    #BMO
    bmo_makespan, bmo_cost, bmo_energy= bmo_optimize(vms, tasks)[:3]
    bmo_mean_makespan+=bmo_makespan
    bmo_mean_cost+=bmo_cost
    bmo_mean_energy+=bmo_energy
    
    #SSA
    ssa_makespan, ssa_cost, ssa_energy= ssa_optimize(vms, tasks)[:3]
    ssa_mean_makespan+=ssa_makespan
    ssa_mean_cost+=ssa_cost
    ssa_mean_energy+=ssa_energy
    
# RR
rr_mean_makespan/=10
rr_mean_energy/=10
rr_mean_cost/=10
print("--RR--",rr_mean_makespan,rr_mean_energy,rr_mean_cost)

#SJF
sjf_mean_makespan/=10
sjf_mean_energy/=10
sjf_mean_cost/=10
print("--SJF--",sjf_mean_makespan,sjf_mean_energy,sjf_mean_cost)

#FWA
fwa_mean_makespan/=10
fwa_mean_energy/=10
fwa_mean_cost/=10
print("--FWA--",fwa_mean_makespan,fwa_mean_energy,fwa_mean_cost)
    
#SQSA
sqsa_mean_makespan/=10
sqsa_mean_energy/=10
sqsa_mean_cost/=10
print("--SQSA--",sqsa_mean_makespan,sqsa_mean_energy,sqsa_mean_cost)

#BAT
bat_mean_makespan/=10
bat_mean_energy/=10
bat_mean_cost/=10
print("--BAT--",bat_mean_makespan,bat_mean_energy,bat_mean_cost)
    
#PSO
pso_mean_makespan/=10
pso_mean_energy/=10
pso_mean_cost/=10
print("--PSO--",pso_mean_makespan,pso_mean_energy,pso_mean_cost)

#BMO
bmo_mean_makespan/=10
bmo_mean_energy/=10
bmo_mean_cost/=10
print("--BMO--",bmo_mean_makespan,bmo_mean_energy,bmo_mean_cost)

#SSA
ssa_mean_makespan/=10
ssa_mean_energy/=10
ssa_mean_cost/=10
print("--SSA--",ssa_mean_makespan,ssa_mean_energy,ssa_mean_cost)

--RR-- 387.0656549366117 2.2171454456240003 221.71454456240008
--SJF-- 391.55685786767924 2.230109462299986 223.01094622999858
--FWA-- 265.4334050330835 1.7690862298899408 176.90862298899407
--SQSA-- 239.0017897544 1.7520538080629584 175.20538080629586
--BAT-- 298.9242726929921 1.8095192803668945 180.95192803668948
--PSO-- 324.4805823930858 1.9791891272494186 197.91891272494186
--BMO-- 271.2272906016891 1.8735041166870818 187.35041166870815
--SSA-- 267.73295473190467 1.8212535188568097 182.125351885681
